In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import search, print_breakdown


df_nhtsa = await search(
  min_model_year=2019,
  include_openpilot=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36764 VINs (filter_comment=None, include_openpilot=True, skipped=163, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/32332 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              996
Bronco Sport       1500
Continental          23
Corsair             986
Ecosport            322
Edge               2450
Escape             3133
Expedition          725
Expedition MAX      629
Explorer           2960
F-150              6926
F-250              1500
F-350              1034
F-450               196
F-550                 1
Fiesta              147
Flex                137
Fusion              908
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             871
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus               40
Transit            1166
Transit Connect     340
dtype: int64


In [2]:
import pandas as pd

from notebooks.ford.platforms import find_openpilot_platform
from notebooks.ford.settings import VehicleSetting, VehicleSettings


parameters = {
  'Wheelbase (ABS)': [VehicleSettings.abs_wheel_base],
  'Wheelbase (APIM)': [VehicleSettings.apim_wheel_base, VehicleSettings.apim_sync4_wheel_base],
  'Wheelbase (IPMA)': [VehicleSettings.ipma_vehicle_cfg_wheelbase],
  'Payload (ABS)': [VehicleSettings.abs_payload],
  'Weight (APIM)': [VehicleSettings.apim_vehicle_weight, VehicleSettings.apim_sync4_vehicle_weight],
  'Steer Ratio (ABS)': [VehicleSettings.abs_steering_gear],
  'Steer Ratio (APIM)': [VehicleSettings.apim_sync4_steering_gear_ratio],
  'Steer Ratio (IPMA)': [VehicleSettings.ipma_vehicle_cfg_steering_ratio],
}

def get_parameter(asbuilt: AsBuiltData, settings: list[VehicleSetting]):
  for setting in settings:
    if not asbuilt.is_present(setting.ecu):
      continue
    return asbuilt.get_setting_value(setting)
  return 'Missing ECU'

def get_parameters(row):
  asbuilt = AsBuiltData.from_vin(row['VIN'])
  return pd.Series({k: get_parameter(asbuilt, v) for k, v in parameters.items()})


df_cars = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear', 'Trim', 'Series', 'ElectrificationLevel']].copy()

df_cars['CarName'] = df_cars['Make'] + ' ' + df_cars['Model'] + ' ' + df_cars['ModelYear'].astype(str)
df_cars['CarInfoPlatform'] = df_cars['CarName'].apply(find_openpilot_platform)

df_cars = df_cars.join(df_cars.apply(get_parameters, axis=1))
df_cars.head()

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


,VIN,Make,Model,ModelYear,Trim,Series,ElectrificationLevel,CarName,CarInfoPlatform,Wheelbase (ABS),Wheelbase (APIM),Wheelbase (IPMA),Payload (ABS),Weight (APIM),Steer Ratio (ABS),Steer Ratio (APIM),Steer Ratio (IPMA)
0,1FMCU9GN2PUB45529,FORD,Escape,2023,,Active,ICE,FORD Escape 2023,None,Unknown (0x00),2.71 m,2.71 m,Base Payload,2000 kg,EPAS 17:1,17:1,15.52
1,3FTTW8E99NRA19557,FORD,Maverick,2022,,SUPERCREW,ICE,FORD Maverick 2022,FORD MAVERICK 1ST GEN,Unknown (0x00),Missing ECU,Missing ECU,Mid Payload Upgrade,Missing ECU,Unknown (0x00),Missing ECU,Missing ECU
2,1FTEW1E5XLKD78382,FORD,F-150,2020,,,ICE,FORD F-150 2020,None,3.68 m,3.68 m,Missing ECU,Base Payload,7000 kg,EPAS 17:1,Missing ECU,Missing ECU
3,1FMCU9H92NUA40763,FORD,Escape,2022,,SEL,ICE,FORD Escape 2022,FORD ESCAPE 4TH GEN,Unknown (0x00),2.69 m,Missing ECU,Base Payload,2100 kg,EPAS 17:1,Missing ECU,Missing ECU
4,1FMCU9BZ0NUA42388,FORD,Escape,2022,,SE,FHEV,FORD Escape 2022,FORD ESCAPE 4TH GEN,Unknown (0x00),2.69 m,Missing ECU,Heavy Duty Payload Upgrade,2100 kg,EPAS 17:1,Missing ECU,Missing ECU


In [3]:
df_parameters = df_cars[~df_cars['CarInfoPlatform'].isna()][['CarInfoPlatform', 'ElectrificationLevel', *parameters.keys()]].copy()

# for parameter in parameters.keys():
#   df_parameters[parameter] = df_parameters[parameter].astype('category')

rows = []
for group in df_parameters.groupby(['CarInfoPlatform', 'ElectrificationLevel', 'Payload (ABS)'], as_index=True):
  print(group[0])
  for parameter in parameters.keys():
    if parameter == 'Payload (ABS)':
      continue
    print(group[1][parameter].value_counts())
    print()
  print()
  print()
  row = dict({'CarInfoPlatform': group[0][0], 'ElectrificationLevel': group[0][1], 'Payload (ABS)': group[0][2]}, **{parameter: sorted(group[1][parameter].unique()) for parameter in parameters.keys() if parameter != 'Payload (ABS)'})
  rows.append(row)

df_platform_parameters = pd.DataFrame(rows)
df_platform_parameters.set_index(['CarInfoPlatform', 'ElectrificationLevel', 'Payload (ABS)'], inplace=True)
df_platform_parameters


(<CAR.BRONCO_SPORT_MK1: 'FORD BRONCO SPORT 1ST GEN'>, 'ICE', 'Base Payload')
Wheelbase (ABS)
Unknown (0x00)    787
Name: count, dtype: int64

Wheelbase (APIM)
2.67 m    786
0.00 m      1
Name: count, dtype: int64

Wheelbase (IPMA)
Missing ECU    787
Name: count, dtype: int64

Weight (APIM)
2100 kg    786
0 kg         1
Name: count, dtype: int64

Steer Ratio (ABS)
EPAS 17:1    787
Name: count, dtype: int64

Steer Ratio (APIM)
Missing ECU    787
Name: count, dtype: int64

Steer Ratio (IPMA)
Missing ECU    787
Name: count, dtype: int64



(<CAR.BRONCO_SPORT_MK1: 'FORD BRONCO SPORT 1ST GEN'>, 'ICE', 'Mid Payload Upgrade')
Wheelbase (ABS)
Unknown (0x00)    143
Name: count, dtype: int64

Wheelbase (APIM)
2.67 m    141
2.69 m      2
Name: count, dtype: int64

Wheelbase (IPMA)
Missing ECU    143
Name: count, dtype: int64

Weight (APIM)
2100 kg    143
Name: count, dtype: int64

Steer Ratio (ABS)
EPAS 17:1    143
Name: count, dtype: int64

Steer Ratio (APIM)
Missing ECU    143
Name: count, dtype

Wheelbase (ABS)  \
CarInfoPlatform             ElectrificationLevel Payload (ABS)                                                  
FORD BRONCO SPORT 1ST GEN   ICE                  Base Payload                                [Unknown (0x00)]   
                                                 Mid Payload Upgrade                         [Unknown (0x00)]   
FORD ESCAPE 4TH GEN         FHEV                 Heavy Duty Payload Upgrade                  [Unknown (0x00)]   
                            ICE                  Base Payload                                [Unknown (0x00)]   
                            PHEV                 Unknown (0x05)                              [Unknown (0x00)]   
FORD EXPLORER 6TH GEN       HEV                  Unknown (0x04)                              [Unknown (0x00)]   
                            ICE                  Base Payload                                [Unknown (0x00)]   
                                                 Heavy Duty Payload Upgrade                  [Unknown (0x00)]   
                                                 Mid Payload Upgrade                         [Unknown (0x00)]   
                            PHEV                 Unknown (0x05)                              [Unknown (0x00)]   
FORD F-150 14TH GEN         BEV                  Base Payload                                        [3.10 m]   
                            HEV                  Base Payload                                [3.68 m, 3.99 m]   
                            ICE                  Base Payload                [3.10 m, 3.58 m, 3.68 m, 3.99 m]   
                                                 Heavy Duty Payload Upgrade                          [3.58 m]   
                                                 Mid Payload Upgrade                                 [3.68 m]   
FORD MAVERICK 1ST GEN       HEV                  Base Payload                                [Unknown (0x00)]   
                                                 Mid Payload Upgrade                         [Unknown (0x00)]   
                            ICE                  Base Payload                                [Unknown (0x00)]   
                                                 Mid Payload Upgrade                         [Unknown (0x00)]   
FORD MUSTANG MACH-E 1ST GEN BEV                  Base Payload                                [Unknown (0x00)]   
                                                 Mid Payload Upgrade                         [Unknown (0x00)]   

                                                                                                     Wheelbase (APIM)  \
CarInfoPlatform             ElectrificationLevel Payload (ABS)                                                          
FORD BRONCO SPORT 1ST GEN   ICE                  Base Payload                                        [0.00 m, 2.67 m]   
                                                 Mid Payload Upgrade                                 [2.67 m, 2.69 m]   
FORD ESCAPE 4TH GEN         FHEV                 Heavy Duty Payload Upgrade                                  [2.69 m]   
                            ICE                  Base Payload                                        [2.69 m, 3.02 m]   
                            PHEV                 Unknown (0x05)                                              [2.69 m]   
FORD EXPLORER 6TH GEN       HEV                  Unknown (0x04)                                              [3.02 m]   
                            ICE                  Base Payload                                                [3.02 m]   
                                                 Heavy Duty Payload Upgrade                          [2.85 m, 3.02 m]   
                                                 Mid Payload Upgrade         [0.00 m, 2.72 m, 2.85 m, 3.02 m, 4.06 m]   
                            PHEV                 Unknown (0x05)                                              [3.02 m]   
FORD F-150 14TH GEN         BEV           